# Setting up a parallel notebook with SLURM, ipyparallel at JSC

The original version of this tutorial was inspired by the [CS228 tutorial](https://github.com/kuleshov/cs228-material/blob/master/tutorials/python/cs228-python-tutorial.ipynb) by Volodomyr Kuleshov and Isaac Caswell.

For this interactive HPC adaptation, we have heavily referenced the [HPC Python](https://gitlab.jsc.fz-juelich.de/sdlbio-courses/hpc-python) course and the [jupyter-jsc](https://github.com/FZJ-JSC/jupyter-jsc-notebooks) repository. Many thanks to Jan Meinke, Jens Henrik Goebbert, Tim Kreuzer, Alice Gorsch @ Jülich Supercomputing Centre for help setting this up.

<div style="float: right; padding-right: 2em; padding-top: 2em;">
    <img src="https://raw.githubusercontent.com/helmholtz-analytics/heat/master/doc/source/_static/images/logo.png"></img>
</div>

## Introduction
<div class="alert alert-block alert-warning">
<b>Note:</b>
This tutorial is designed to run on <a href="https://jupyter-jsc.fz-juelich.de/">Jupyter-JSC</a>, a JupyterLab environment provided by the Jülich Supercomputing Centre.  
If you want to run the tutorial on your local machine, please refer to the `tutorials/local` directory in this repository.
</div>




## Setting up the environment

The rest of this tutorial assumes you have started a JupyterLab at [jupyter-jsc](https://jupyter-jsc.fz-juelich.de/) with the following parameters:

| **Lab Config** |  |
| --- | --- |
| Name | [YOUR CHOICE] |
| Version | JupyterLab - 3.6|
| System | JURECA |
| Account | [YOUR USERNAME] |
| Project | training2404 |
| Partition | dc-gpu |
| Reservation | derse24_heat |

| **Resources** | |
| -- | -- |
| Nodes [1,24] | 1 |
| GPUs [1,4] | 4 |
| Runtime (minutes) | 90 |

The `derse24_heat` reservation is available from 13:30 to 15:00 on the 6th of March, 2024.

It might take several minutes for the JupyterLab to start. Once it does, the JupyterLab interface will open in your browser with a *Launcher* tab.

Scroll down to find the *Other* section and start the *Terminal*.

In the terminal, copy the tutorials from the project directory to your home directory:

```bash
cd
cp -r /p/project/training2404/tutorials* .
```

Now you have your copy of the tutorials and can modify them as you like. 

Navigate to `$HOME`, then `tutorials/hpc`. Open `1_intro.ipynb`.

You will be prompted to select a kernel. Choose `heat-1.4.0-dev`, which includes all features on the Heat `main` branch as of March 4th 2024.

You can access the `heat-1.4.0-dev` virtual environment from the terminal by typing:

```bash
source /p/project/training2404/heat_derse24.sh
```

In the terminal, you can run the following command to see what packages (modules) are part of the kernel environment:

```bash
module list
```



### Resources

We will be running the tutorial on the GPU partition of the [JURECA](https://apps.fz-juelich.de/jsc/hps/jureca/configuration.html) cluster, with the following hardware:

- 2× AMD EPYC 7742, 2× 64 cores, 2.25 GHz
- 4× NVIDIA A100 GPU, 4× 40 GB HBM2e


Each of you has access to 1 node and 4 GPUs.

To be able to run this tutorial interactively for parallel computing, we need to start an [IPython cluster](https://ipyparallel.readthedocs.io/en/latest/tutorial/process.html).


In the terminal, type:

```bash
ipcontroller  &
srun -n 4 -c 12 --ntasks-per-node 4 --time 00:90:00   -A training2404 -p dc_gpu ipengine start
```
On your terminal, you should see something like this:

```bash
2024-03-04 16:30:24.740 [IPController] Registering 4 new hearts
2024-03-04 16:30:24.740 [IPController] registration::finished registering engine 0:63ac2343-f1deab70b14c0e14ca4c1630 in 5672ms
2024-03-04 16:30:24.740 [IPController] engine::Engine Connected: 0
2024-03-04 16:30:24.744 [IPController] registration::finished registering engine 3:673ce83c-eb7ccae6c69c52382c8349c1 in 5397ms
2024-03-04 16:30:24.744 [IPController] engine::Engine Connected: 3
2024-03-04 16:30:24.745 [IPController] registration::finished registering engine 1:d7936040-5ab6c117b845850a3103b2e8 in 5627ms
2024-03-04 16:30:24.745 [IPController] engine::Engine Connected: 1
2024-03-04 16:30:24.745 [IPController] registration::finished registering engine 2:ca57a419-2f2c89914a6c17865103c3e7 in 5508ms
2024-03-04 16:30:24.745 [IPController] engine::Engine Connected: 2
```

<div class="alert alert-block alert-info">
<b>Note:</b>
You must now reload the kernel to be able to access the IPython cluster.
</div>


## What is Heat for?

[**deRSE24 NOTE**:  do attend Fabian Hoppe's talk [TODAY at 16:30](https://events.hifis.net/event/994/contributions/7940/) for more details, benchmarks, and an overview of the parallel Python ecosystem.] 


Straight from our [GitHub repository](https://github.com/helmholtz-analytics/heat):

Heat builds on [PyTorch](https://pytorch.org/) and [mpi4py](https://mpi4py.readthedocs.io) to provide high-performance computing infrastructure for memory-intensive applications within the NumPy/SciPy ecosystem.


With Heat you can:
- port existing NumPy/SciPy code from single-CPU to multi-node clusters with minimal coding effort;
- exploit the entire, cumulative RAM of your many nodes for memory-intensive operations and algorithms;
- run your NumPy/SciPy code on GPUs (CUDA, ROCm, coming up: Apple MPS).


Why?

- significant **scalability** with respect to task-parallel frameworks;
- analysis of massive datasets without breaking them up in artificially independent chunks;
- ease of use: script and test on your laptop, port straight to HPC cluster; 
- PyTorch-based: GPU support beyond the CUDA ecosystem.

<div>
  <img src=https://github.com/helmholtz-analytics/heat/blob/docs/tutorials-hpc/doc/source/_static/images/heatvsdask_strong_smalldata_without.png?raw=true title="Strong scaling CPU" width="30%" style="float:center"/>
  <img src=https://github.com/helmholtz-analytics/heat/blob/docs/tutorials-hpc/doc/source/_static/images/heatvsdask_weak_smalldata_without.png?raw=true title="Weak scaling CPU" width="30%" style="float:center "/>
  <img src=https://github.com/helmholtz-analytics/heat/blob/docs/tutorials-hpc/doc/source/_static/images/weak_scaling_gpu_terrabyte.png?raw=true title="Weak scaling GPU" width="30%" style="float:center"/>
</div>

To be able to start working with Heat on an HPC cluster, we first need to check the health of the available processes. We will use `ipyparallel` for this. For a great intro on `ipyparallel` usage on our supercomputers, check out Jan Meinke's tutorial ["Interactive Parallel Computing with IPython Parallel"](https://gitlab.jsc.fz-juelich.de/sdlbio-courses/hpc-python/-/blob/master/06_LocalParallel.ipynb) or the [ipyparallel docs](https://ipyparallel.readthedocs.io/en/latest/).

In [ ]:
from ipyparallel import Client
rc = Client(profile="default")

Earlier, we have started an IPython cluster with 4 processes. We can now check if the processes are available.

In [ ]:
rc.ids

[0, 1, 2, 3]

The `px` magic command allows you to execute Python commands or a Jupyter cell on the ipyparallel engines interactively ([%%px documentation](https://ipyparallel.readthedocs.io/en/latest/tutorial/magics.html)).

We can now finally import `heat` on our 4-process cluster.

In [ ]:
%px import heat as ht